In [1]:
# basic imports
import tensorflow as tf 

import baseline_config

In [2]:
# load the pre-processed dataset
dataset = tf.keras.utils.image_dataset_from_directory(baseline_config.dataset_path, labels='inferred')

Found 0 files belonging to 5 classes.


ValueError: No images found in directory c:/birdclef2022/. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')